In [1]:
import os

import pandas as pd
import netCDF4 as nc
import numpy as np

In [2]:
DATA_FILE_DIR = './data/'

START_YEAR, END_YEAR = 2010, 2020

NUM_OF_YEARS = END_YEAR - START_YEAR

NUM_OF_MONTHS = 12

NUM_OF_DAYS = {1: 31, 2: 28, 3: 31, 4: 30, 5: 31, 6: 30, 7: 31, 8: 31, 9: 30, 10: 31, 11: 30, 12: 31,}

In [36]:
file = nc.Dataset('./data/20110101.nc4')

In [37]:
lat = file.variables['lat'][:].filled()
lon = file.variables['lon'][:].filled()
tmp = file.variables['AvgSurfT_tavg'][0]
num_of_loc = tmp.size - tmp.mask.sum()
file.close()

In [39]:
LAT = lat.size
LON = lon.size

In [132]:
locations = np.zeros(shape=(num_of_loc, 2))
count = 0
for i in range(LAT):
    for j in range(LON):
        if not tmp.mask[i, j]:
            locations[count] = [lat[i], lon[j]]
            count += 1
yearly_mean = pd.DataFrame(locations, columns=['lat', 'lon'], dtype='float32')

In [133]:
def get_tmp(filepath):
    assert os.path.isfile(filepath), '{} does not exist!'.format(filepath)
    
    file = nc.Dataset(filepath)
    temperature = file.variables['AvgSurfT_tavg'][0]
    file.close()
    return temperature.filled(np.nan)

In [134]:
%%time
date_temp = pd.DataFrame()
for month in range(1, NUM_OF_MONTHS+1):
    for day in range(1, NUM_OF_DAYS[month]+1):
        date = '{:02d}{:02d}'.format(month, day)
        print(date)
        temps = np.ndarray(shape=(NUM_OF_YEARS, LAT, LON))
        for year in range(START_YEAR, END_YEAR):
            day = str(year) + date
            filepath = DATA_FILE_DIR + day + '.nc4'
            temps[year-START_YEAR] = get_tmp(filepath)
            
        mean_temps = np.nanmean(temps, axis=0)
        mean_temps = mean_temps.reshape(-1)
        date_temp[date] = mean_temps[~np.isnan(mean_temps)]

date_temp -= 273.15
date_temp = date_temp.round(decimals=1)

yearly_mean = pd.concat([yearly_mean, date_temp], axis=1)

0101


<timed exec>:12: RuntimeWarning: Mean of empty slice


0102
0103
0104
0105
0106
0107
0108
0109
0110
0111
0112
0113
0114
0115
0116
0117
0118
0119
0120
0121
0122
0123
0124
0125
0126
0127
0128
0129
0130
0131
0201
0202
0203
0204
0205
0206
0207
0208
0209
0210
0211
0212
0213
0214
0215
0216
0217
0218
0219
0220
0221
0222
0223
0224
0225
0226
0227
0228
0301
0302
0303
0304
0305
0306
0307
0308
0309
0310
0311
0312
0313
0314
0315
0316
0317
0318
0319
0320
0321
0322
0323
0324
0325
0326
0327
0328
0329
0330
0331
0401
0402
0403
0404
0405
0406
0407
0408
0409
0410
0411
0412
0413
0414
0415
0416
0417
0418
0419
0420
0421
0422
0423
0424
0425
0426
0427
0428
0429
0430
0501
0502
0503
0504
0505
0506
0507
0508
0509
0510
0511
0512
0513
0514
0515
0516
0517
0518
0519
0520
0521
0522
0523
0524
0525
0526
0527
0528
0529
0530
0531
0601
0602
0603
0604
0605
0606
0607
0608
0609
0610
0611
0612
0613
0614
0615
0616
0617
0618
0619
0620
0621
0622
0623
0624
0625
0626
0627
0628
0629
0630
0701
0702
0703
0704
0705
0706
0707
0708
0709
0710
0711
0712
0713
0714
0715
0716
0717
0718
0719
0720


In [136]:
os.makedirs('./processed_mean/', exist_ok=True)
yearly_mean.to_csv('./processed_mean/processed_yearly_avg.csv', index=False)